In [40]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv

load_dotenv()


True

In [41]:
token2 = os.getenv("token2") 

In [42]:
title = 'The Bang Bang Club'
year = ''
res = requests.get('http://www.omdbapi.com/?apikey='+token2+'&t='+title+'&y='+year).json()
res

{'Title': 'The Bang Bang Club',
 'Year': '2010',
 'Rated': 'R',
 'Released': '22 Jul 2011',
 'Runtime': '106 min',
 'Genre': 'Biography, Drama, History',
 'Director': 'Steven Silver',
 'Writer': 'Steven Silver, Greg Marinovich, João Silva',
 'Actors': 'Ryan Phillippe, Malin Akerman, Taylor Kitsch',
 'Plot': 'A drama based on the true-life experiences of four combat photographers capturing the final days of apartheid in South Africa.',
 'Language': 'English, Zulu, Xhosa, Afrikaans',
 'Country': 'Canada, South Africa',
 'Awards': '13 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjIxNDIyMDU2Ml5BMl5BanBnXkFtZTcwODUxMTA4NA@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '49%'},
  {'Source': 'Metacritic', 'Value': '48/100'}],
 'Metascore': '48',
 'imdbRating': '6.9',
 'imdbVotes': '10,971',
 'imdbID': 'tt1173687',
 'Type': 'movie',
 'DVD': '16 Aug 2011',
 'BoxOffice': 'N/A',
 'Production': 

In [43]:
df = pd.read_csv('top_4000_movies_data.csv')

In [44]:
df.head()

,Release Date,Movie Title,Production Budget,Domestic Gross,Worldwide Gross
0,4/23/2019,Avengers: Endgame,400000000,858373000,2797800564
1,5/20/2011,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802
2,4/22/2015,Avengers: Age of Ultron,365000000,459005868,1395316979
3,12/16/2015,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2064615817
4,4/25/2018,Avengers: Infinity War,300000000,678815482,2044540523


In [45]:
df.columns = df.columns.str.replace(' ', '_')

In [46]:
null_counts = df.isnull().sum()
print (null_counts)

Release_Date         21
Movie_Title           0
Production_Budget     0
Domestic_Gross        0
Worldwide_Gross       0
dtype: int64


In [47]:
df = df.dropna()

In [48]:
df['Release_Date'] = pd.to_datetime(df['Release_Date'], format='%m/%d/%Y')

In [49]:
df['Years'] = df['Release_Date'].dt.year
df.head()

,Release_Date,Movie_Title,Production_Budget,Domestic_Gross,Worldwide_Gross,Years
0,2019-04-23,Avengers: Endgame,400000000,858373000,2797800564,2019
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802,2011
2,2015-04-22,Avengers: Age of Ultron,365000000,459005868,1395316979,2015
3,2015-12-16,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2064615817,2015
4,2018-04-25,Avengers: Infinity War,300000000,678815482,2044540523,2018


In [61]:
df_sample=df.head(50)
df_sample

,Release_Date,Movie_Title,Production_Budget,Domestic_Gross,Worldwide_Gross,Years
0,2019-04-23,Avengers: Endgame,400000000,858373000,2797800564,2019
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,379000000,241071802,1045713802,2011
2,2015-04-22,Avengers: Age of Ultron,365000000,459005868,1395316979,2015
3,2015-12-16,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2064615817,2015
4,2018-04-25,Avengers: Infinity War,300000000,678815482,2044540523,2018
5,2007-05-24,Pirates of the Caribbean: At World’s End,300000000,309420425,960996492,2007
6,2017-11-13,Justice League,300000000,229024295,655945209,2017
7,2015-10-06,Spectre,300000000,200074175,879500760,2015
8,2019-12-18,Star Wars: The Rise of Skywalker,275000000,515202542,1072848487,2019
9,2018-05-23,Solo: A Star Wars Story,275000000,213767512,393151347,2018


In [52]:
df_sample['Years'] = df_sample['Years'].astype(str)

C:\Users\smico\AppData\Local\Temp\ipykernel_18144\3210469278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['Years'] = df_sample['Years'].astype(str)


In [53]:
df_sample.dtypes

Release_Date         datetime64[ns]
Movie_Title                  object
Production_Budget             int64
Domestic_Gross                int64
Worldwide_Gross               int64
Years                        object
dtype: object

In [54]:
titulos=list(df_sample.Movie_Title)
años=list(df_sample.Years)

In [63]:
titulo_año=zip(titulos,años)

In [55]:
def request_ombd(titulo): 
    
    token2 = os.getenv("token2") 
    
    
    return requests.get('http://www.omdbapi.com/?apikey='+token2+'&t='+title+'&y='+year).json()
    
    

In [56]:
resultado = []
for titulos in df_sample["Movie_Title"]:
    resultado.append(request_ombd(titulos))


[{'Title': "Gospod postoi, imeto i' e Petrunija",
  'Year': '2019',
  'Rated': 'N/A',
  'Released': '25 Jun 2021',
  'Runtime': '100 min',
  'Genre': 'Drama',
  'Director': 'Teona Strugar Mitevska',
  'Writer': 'Teona Strugar Mitevska, Elma Tataragic',
  'Actors': 'Zorica Nusheva, Labina Mitevska, Stefan Vujisic',
  'Plot': 'A Macedonian woman throws herself into a traditionally men-only ceremony, kicking up a ruckus and standing her ground.',
  'Language': 'Macedonian',
  'Country': 'N/A',
  'Awards': '8 wins & 12 nominations',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNWUwOGU4ZTYtMDJiMy00YTg2LTg4OGItZDVlNDU4NDc4ZWUxXkEyXkFqcGdeQXVyMjI5NjkxODA@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.8/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '80%'},
   {'Source': 'Metacritic', 'Value': '50/100'}],
  'Metascore': '50',
  'imdbRating': '6.8',
  'imdbVotes': '2,304',
  'imdbID': 'tt8054608',
  'Type': 'movie',
  'DVD': 'N/A',
  'BoxOffice': 'N/A'

In [57]:
resultado [0]

{'Title': "Gospod postoi, imeto i' e Petrunija",
 'Year': '2019',
 'Rated': 'N/A',
 'Released': '25 Jun 2021',
 'Runtime': '100 min',
 'Genre': 'Drama',
 'Director': 'Teona Strugar Mitevska',
 'Writer': 'Teona Strugar Mitevska, Elma Tataragic',
 'Actors': 'Zorica Nusheva, Labina Mitevska, Stefan Vujisic',
 'Plot': 'A Macedonian woman throws herself into a traditionally men-only ceremony, kicking up a ruckus and standing her ground.',
 'Language': 'Macedonian',
 'Country': 'N/A',
 'Awards': '8 wins & 12 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNWUwOGU4ZTYtMDJiMy00YTg2LTg4OGItZDVlNDU4NDc4ZWUxXkEyXkFqcGdeQXVyMjI5NjkxODA@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '80%'},
  {'Source': 'Metacritic', 'Value': '50/100'}],
 'Metascore': '50',
 'imdbRating': '6.8',
 'imdbVotes': '2,304',
 'imdbID': 'tt8054608',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': 'N/A',
 'Production': 'N/A',


In [62]:
peliculas = pd.DataFrame(resultado)
peliculas

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,totalSeasons
0,"Gospod postoi, imeto i' e Petrunija",2019,N/A,25 Jun 2021,100 min,Drama,Teona Strugar Mitevska,"Teona Strugar Mitevska, Elma Tataragic","Zorica Nusheva, Labina Mitevska, Stefan Vujisic",A Macedonian woman throws herself into a tradi...,...,6.8,"2,304",tt8054608,movie,N/A,N/A,N/A,N/A,True,NaN
1,S,2019,N/A,N/A,15 min,"Short, Family",Hamed Aslani,Hamed Aslani,Sam Valipour,A 4-year-old boy is investigated by the police...,...,N/A,N/A,tt10464882,movie,N/A,N/A,N/A,N/A,True,NaN
2,Creem: America's Only Rock 'n' Roll Magazine,2019,TV-MA,07 Aug 2020,75 min,Documentary,Scott Crawford,"Scott Crawford, Jaan Uhelszki","Jeff Ament, Lester Bangs, Alice Cooper",Explores the seminal music magazine from its 1...,...,6.9,284,tt7808498,movie,27 Aug 2020,N/A,N/A,N/A,True,NaN
3,S,2019,N/A,N/A,15 min,"Short, Family",Hamed Aslani,Hamed Aslani,Sam Valipour,A 4-year-old boy is investigated by the police...,...,N/A,N/A,tt10464882,movie,N/A,N/A,N/A,N/A,True,NaN
4,Surviving R. Kelly,2019,TV-MA,03 Jan 2019,306 min,"Documentary, Crime, Music",N/A,N/A,"Andrea Kelly, Tarana Burke, Touré",A six-part series about the controversial R&am...,...,7.5,"5,376",tt8385496,series,NaN,NaN,NaN,NaN,True,1
5,D-Day: Battle of Omaha Beach,2019,Not Rated,13 Sep 2019,85 min,"Action, Drama, History",Nick Lyon,Geoff Meed,"Randy Couture, Chuck Liddell, Weston Cage",When an elite group of American soldiers are o...,...,3.8,"1,224",tt10239930,movie,N/A,N/A,N/A,N/A,True,NaN
6,"Gospod postoi, imeto i' e Petrunija",2019,N/A,25 Jun 2021,100 min,Drama,Teona Strugar Mitevska,"Teona Strugar Mitevska, Elma Tataragic","Zorica Nusheva, Labina Mitevska, Stefan Vujisic",A Macedonian woman throws herself into a tradi...,...,6.8,"2,304",tt8054608,movie,N/A,N/A,N/A,N/A,True,NaN
7,"Gospod postoi, imeto i' e Petrunija",2019,N/A,25 Jun 2021,100 min,Drama,Teona Strugar Mitevska,"Teona Strugar Mitevska, Elma Tataragic","Zorica Nusheva, Labina Mitevska, Stefan Vujisic",A Macedonian woman throws herself into a tradi...,...,6.8,"2,304",tt8054608,movie,N/A,N/A,N/A,N/A,True,NaN
8,Surviving R. Kelly,2019,TV-MA,03 Jan 2019,306 min,"Documentary, Crime, Music",N/A,N/A,"Andrea Kelly, Tarana Burke, Touré",A six-part series about the controversial R&am...,...,7.5,"5,376",tt8385496,series,NaN,NaN,NaN,NaN,True,1
9,"Yo, mi mujer y mi mujer muerta",2019,N/A,21 Mar 2019,97 min,"Comedy, Drama",Santi Amodeo,"Santi Amodeo, Rafael Cobos","Oscar Martínez, Carlos Areces, Ingrid García J...",An Argentinian professor and architect agrees ...,...,5.1,242,tt7893296,movie,09 May 2020,N/A,N/A,N/A,True,NaN


In [60]:
peliculas.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response',
       'totalSeasons'],
      dtype='object')